# NEOM Potential Recharge Zones (region swale potential)

Amgad, James, Josh

Mapping NEOM region for suitability of building features (such as swales) to capture surface/ sub-surface water, mainly to build up plant-available water in the soil.

Layers/ Factors:
1. Slope : Restricted to [0-10 degrees], gradual slopes are more desirable
2. Top Soil Types: Medium texture (loam/ silt) score highest, second is fine clays, and last is coarse sands for retaining plant-avaialble water
3. Runoff: Higher runoff is better, masked to values only above 1E-3 mm per grid box

## Preliminaries

In [1]:
import ee
import numpy as np

from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm

from statistics import mean
import folium
from folium import plugins
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d

Loading BokehJS ...

In [2]:
ee.Initialize()

bbox_name = 'neom'
bbox = common.bboxes()[bbox_name]

## Predictors 
**Terrain Slope** from [Merit Hydro](https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description)

In [31]:
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope').clip(bbox)
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_inv = slope_img.updateMask(slope_mask).pow(-1) 
slope_img_scaled = norm.img_scale(slope_img_inv, area_of_interest=bbox) 
norm.img_range(slope_img_scaled, area_of_interest=bbox)

[0, 1]

**Soil Types** qualitative ranking.

In [32]:
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").clip(bbox)

#categorizing soil types at 0, 10, and 30 cm depths based on retaining plant-available water ability
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 5) ? 1.0" +
        ": (b('b0') == 7) ? 1.0" +
        ": (b('b0') == 8) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 2) ? 0.7" +
        ": (b('b0') == 4) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 9) ? 0.7" +
        ": (b('b0') == 1) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 11) ? 0.4" +
        ": (b('b0') == 12) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 5) ? 1.0" +
        ": (b('b10') == 7) ? 1.0" +
        ": (b('b10') == 8) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 2) ? 0.7" +
        ": (b('b10') == 4) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 9) ? 0.7" +
        ": (b('b10') == 1) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 11) ? 0.4" +
        ": (b('b10') == 12) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 5) ? 1.0" +
        ": (b('b30') == 7) ? 1.0" +
        ": (b('b30') == 8) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 2) ? 0.7" +
        ": (b('b30') == 4) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 9) ? 0.7" +
        ": (b('b30') == 1) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 11) ? 0.4" +
        ": (b('b30') == 12) ? 0.4" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

In [33]:
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=bbox)
norm.img_range(top_soils_scaled, area_of_interest=bbox)

[0, 1]

In [35]:
#importing runoff for January, masking for values higher than 1e-03 mm
runoff_clim = ee.ImageCollection('users/jamesmcc/era5_sfc_runoff_neom_climatology')
runoff = runoff_clim.sum().clip(bbox)
runoff_mask = runoff.gte(1e-06)
runoff_update = runoff.updateMask(runoff_mask)
runoff_scaled = norm.img_scale(runoff_update, area_of_interest=bbox)
norm.img_range(runoff_scaled, area_of_interest=bbox)

[0, 1]

In [36]:
#Compiling score, printing score range
score = (runoff_scaled.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score = norm.img_scale(score, area_of_interest=bbox)
norm.img_range(score, area_of_interest=bbox)

[0, 0.9999999999999999]

In [37]:
# Plot a histogram of the scores
runoff_hist = ee.Dictionary(
    score.clip(bbox)
    .reduceRegion(
        ee.Reducer.histogram()).get('surface_runoff'))

hist = np.array(runoff_hist.get('histogram').getInfo())
edges = (
    (np.array(list(range(len(hist) + 1))) * 
     np.array(runoff_hist.get('bucketWidth').getInfo())) + 
    np.array(runoff_hist.get('bucketMin').getInfo()) )

p = figure(
    title='NEOM PRZ Score', 
    background_fill_color="white", height = 300)
_ = p.quad(
    top=hist,
    bottom=1,
    left=edges[:-1],
    right=edges[1:],
    fill_color="navy",
    line_color="navy",
    alpha=1.)
show(p)

In [60]:
# Map parameters
box_corners = common.bboxes()['neom'].toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

palette_name = 'Blues'
palette_len = 5

vis_min = 0
vis_max = 1

palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=vis_min, maximum=vis_max, title='NEOM PRZ Score')

vis_params = {
    'min': vis_min, 'max': vis_max, 'dimensions': 512,
    'palette': palette}

In [87]:
#plotting score for NEOM region
the_map = vis.folium_map(location=[center_lat-1.1, center_lon], zoom_start=9, height=1500)
the_map.add_ee_layer(score, vis_params, name = 'NEOM PRZ Score')
vis.folium_display(the_map)

In [95]:
#calculate sum of potential water storage from neom area over a year
def sum_runoff(runoff_img, geometry, scale):
    return (
        runoff_img
        .reduceRegion(reducer= ee.Reducer.sum(), geometry=geometry, scale=100)
        .getInfo()['surface_runoff']*(scale*scale))

def sum_runoff_score_thresh(runoff_img, score, thresh, geometry, scale):
  thresh_mask = score.gte(thresh)
  return sum_runoff(runoff_img.updateMask(thresh_mask), geometry, scale)

In [92]:
potential_recharge = sum_runoff(runoff, bbox, 100)
thresh_recharge = [sum_runoff_score_thresh(runoff, score, thresh, bbox, 100) for thresh in [0, .2, .4, .6, .8]]